### Name:
> Castillo, Marvien Angel C. <br>
> Herrera, Mikhaela Gabrielle B. <br>
> Regindin, Sean Adrien I. <br>

# Setup Environment

In [1]:
import os

# Add the directory containing the executable to the PATH
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"

# Check if the directory is added to the PATH
print(os.environ["PATH"])

/opt/tljh/user/bin:/bin:/usr/bin:/usr/local/cuda/bin


# Check if CUDA is present

In [2]:
%%bash
nvcc --version
nvprof --version
nsys --version
ncu --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Wed_Apr__9_19:24:57_PDT_2025
Cuda compilation tools, release 12.9, V12.9.41
Build cuda_12.9.r12.9/compiler.35813241_0
nvprof: NVIDIA (R) Cuda command line profiler
Copyright (c) 2012 - 2025 NVIDIA Corporation
Release version 12.9.19 (21)
NVIDIA Nsight Systems version 2025.1.3.140-251335620677v0
NVIDIA (R) Nsight Compute Command Line Profiler
Copyright (c) 2018-2025 NVIDIA Corporation
Version 2025.2.0.0 (build 35613519) (public-release)


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov  6 06:01:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.51.03              Driver Version: 575.51.03      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-32GB           Off |   00000000:00:10.0 Off |                    0 |
| N/A   27C    P0             22W /  250W |       0MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Variant 1 - C Program

In [4]:
%%writefile C_var1.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
// ***C function version
void kernel_C(float A[], float B[], float C[], size_t n, int idx[]) {
	for (int i = 0; i < n; i++) {
		if (A[i] >= B[i]) {
			C[i] = A[i];
			idx[i] = 0;
		}
		else {
			C[i] = B[i];
			idx[i] = 1;
		}
	}
}

int main(int argc, char** argv)
{
   const size_t ARRAY_SIZE = 1<<24;
   const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
   const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
//number of times the program is to be executed
   const size_t loope = 10;
//declare array
   float *C,*A,*B;
   int *idx,a;
   A = (float*)malloc(FLOAT_ARRAY_BYTES);
   B = (float*)malloc(FLOAT_ARRAY_BYTES);
   C = (float*)malloc(FLOAT_ARRAY_BYTES);
   idx = (int*)malloc(INT_ARRAY_BYTES);
   a=2;
//timer variables
  clock_t start, end;
// ***--- initialize your array here ---------
   int i;
	for (i = 0; i < ARRAY_SIZE; i++) {
		A[i] = sin(i * 0.0005) * 100.0 + 50.0;
		B[i] = cos(i * 0.0003) * 100.0 + 50.0;
	}
// fill-in cache
    kernel_C(A,B,C,ARRAY_SIZE,idx);
//time here
  double elapse, time_taken;
  elapse = 0.0f;
  for (int i=0; i<loope; i++){
    start = clock();
      kernel_C(A,B,C,ARRAY_SIZE,idx );
    end = clock();
    time_taken = ((double)(end-start))*1E3/CLOCKS_PER_SEC;
    elapse = elapse + time_taken;
  }
  printf("Function (in C) average time for %lu loops is %f milliseconds to execute an array size %lu \n", loope, elapse/loope, ARRAY_SIZE);

// error checking routine here --
   size_t err_count = 0;
    for (int i=0; i<ARRAY_SIZE; i++){
        if(idx[i] == 0) {
            if(C[i] != A[i]) {
                err_count++;
            }
        }
        else if (idx[i] == 1) {
            if(C[i] != B[i]) {
                err_count++;
            }
        }
    }
   printf("Error count (C program): %lu\n", err_count);
  // Free memory
  free(A);
  free(B);
  free(C);
  free(idx);
  return 0;
}

Overwriting C_var1.c


In [5]:
%%bash
gcc C_var1.c -o C_var1 -lm

In [6]:
%%bash
./C_var1

Function (in C) average time for 10 loops is 241.238400 milliseconds to execute an array size 16777216 
Error count (C program): 0


# Variant 2 - Grid Stride Loop

In [7]:
%%writefile CUDA_var2.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
//Grid stride loop

//*** CUDA kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);
// *** init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }

// *** setup CUDA kernel
  size_t numThreads = 1024;
  size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function = Float kernel_C\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks,numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize();
//error checking
    size_t err_count = 0;
    for (int i=0; i<ARRAY_SIZE; i++){
        if(idx[i] == 0) {
            if(C[i] != A[i]) {
                err_count++;
            }
        }
        else if (idx[i] == 1) {
            if(C[i] != B[i]) {
                err_count++;
            }
        }
    }
  printf("Error count(CUDA program): %zu\n", err_count);
//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;
}


Overwriting CUDA_var2.cu


In [8]:
%%bash
nvcc CUDA_var2.cu -o CUDA_var2 -Wno-deprecated-gpu-targets

In [9]:
%%bash
nvprof ./CUDA_var2

==1024212== NVPROF is profiling process 1024212, command: ./CUDA_var2


*** function = Float kernel_C
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Error count(CUDA program): 0


==1024212== Profiling application: ./CUDA_var2
==1024212== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  104.62ms        10  10.462ms  343.84us  101.52ms  kernel_C(unsigned long, float*, float*, float*, int*)
      API calls:   91.47%  1.46767s         4  366.92ms  49.380us  1.46663s  cudaMallocManaged
                    6.51%  104.52ms         1  104.52ms  104.52ms  104.52ms  cudaDeviceSynchronize
                    1.82%  29.226ms         4  7.3065ms  6.2992ms  9.2770ms  cudaFree
                    0.15%  2.3799ms        10  237.99us  9.3640us  2.2190ms  cudaLaunchKernel
                    0.03%  463.40us       114  4.0640us     114ns  204.80us  cuDeviceGetAttribute
                    0.01%  192.44us         1  192.44us  192.44us  192.44us  cuDeviceGetName
                    0.00%  38.228us         1  38.228us  38.228us  38.228us  cuDeviceTotalMem
                    0.00%  14.452us         1  14.4

# Variant 3.0 - Grid Stride Loop with Prefetch

In [10]:
%%writefile CUDA_var3.cu
// prefetch
#include <stdio.h>
#include <stdlib.h>

//CUDA kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);

// ****init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }
 //"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// setup CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize(); 

//"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,cudaCpuDeviceId,NULL);

//error checking
    size_t err_count = 0;
    for (int i=0; i<ARRAY_SIZE; i++){
        if(idx[i] == 0) {
            if(C[i] != A[i]) {
                err_count++;
            }
        }
        else if (idx[i] == 1) {
            if(C[i] != B[i]) {
                err_count++;
            }
        }
    }
  printf("Error count(CUDA program): %zu\n", err_count);

//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;
}

Overwriting CUDA_var3.cu


In [11]:
%%bash
nvcc CUDA_var3.cu -o CUDA_var3 -Wno-deprecated-gpu-targets

In [12]:
%%bash
nvprof ./CUDA_var3

==1024270== NVPROF is profiling process 1024270, command: ./CUDA_var3


*** function ***
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Error count(CUDA program): 0


==1024270== Profiling application: ./CUDA_var3
==1024270== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.5445ms        10  354.45us  350.24us  359.36us  kernel_C(unsigned long, float*, float*, float*, int*)
      API calls:   86.79%  1.14978s         4  287.45ms  68.581us  1.14878s  cudaMallocManaged
                   10.33%  136.83ms         8  17.104ms  15.562us  44.159ms  cudaMemPrefetchAsync
                    1.37%  18.107ms         4  4.5267ms  3.5474ms  5.4832ms  cudaFree
                    1.21%  15.976ms        10  1.5976ms  9.3090us  15.825ms  cudaLaunchKernel
                    0.25%  3.3265ms         1  3.3265ms  3.3265ms  3.3265ms  cudaDeviceSynchronize
                    0.04%  498.20us       114  4.3700us     139ns  194.46us  cuDeviceGetAttribute
                    0.02%  206.83us         1  206.83us  206.83us  206.83us  cuDeviceGetName
                    0.01%  70.355us         1  

# Variant 4.0 - Grid Stride Loop with Prefetch and Page Creation

In [13]:
%%writefile CUDA_var4.cu
//prefetch + page creation
// page creation responsible gpu fault page
#include <stdio.h>
#include <stdlib.h>

//CUDA kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}


int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);
//get gpu id
  int device = -1;
  cudaGetDevice(&device);

//"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
//"prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// ****init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }
//"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// setup CUDA kernel
    size_t numThreads = 1024;   
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize();

  //"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,cudaCpuDeviceId,NULL);

//error checking
    size_t err_count = 0;
    for (int i=0; i<ARRAY_SIZE; i++){
        if(idx[i] == 0) {
            if(C[i] != A[i]) {
                err_count++;
            }
        }
        else if (idx[i] == 1) {
            if(C[i] != B[i]) {
                err_count++;
            }
        }
    }
  printf("Error count(CUDA program): %zu\n", err_count);


//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;
}

Overwriting CUDA_var4.cu


In [14]:
%%bash
nvcc CUDA_var4.cu -o CUDA_var4 -Wno-deprecated-gpu-targets

In [15]:
%%bash
nvprof ./CUDA_var4

==1024327== NVPROF is profiling process 1024327, command: ./CUDA_var4


*** function ***
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Error count(CUDA program): 0


==1024327== Profiling application: ./CUDA_var4
==1024327== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.6407ms        10  364.07us  362.27us  369.63us  kernel_C(unsigned long, float*, float*, float*, int*)
      API calls:   71.86%  1.12144s         4  280.36ms  79.531us  1.12039s  cudaMallocManaged
                   25.56%  398.93ms        12  33.244ms  18.953us  96.565ms  cudaMemPrefetchAsync
                    1.28%  19.922ms         4  4.9805ms  4.5466ms  5.3526ms  cudaFree
                    1.01%  15.815ms        10  1.5815ms  7.2500us  15.642ms  cudaLaunchKernel
                    0.22%  3.4303ms         1  3.4303ms  3.4303ms  3.4303ms  cudaDeviceSynchronize
                    0.04%  643.68us       114  5.6460us     102ns  267.05us  cuDeviceGetAttribute
                    0.01%  205.81us         1  205.81us  205.81us  205.81us  cuDeviceGetName
                    0.00%  62.119us         1  

# Variant 5.0 - Grid Stride Loop with Prefetch and Page Creation + mem advise

In [16]:
%%writefile CUDA_var5.cu
//prefetch + page creation + memadvise

#include <stdio.h>
#include <stdlib.h>

//CUDA kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);

// memory advise
   cudaMemAdvise(A, FLOAT_ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
   cudaMemAdvise(A, FLOAT_ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
   cudaMemAdvise(B, FLOAT_ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
   cudaMemAdvise(B, FLOAT_ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

//"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
//"prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// ****init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }


 //"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// setup CUDA kernel
    size_t numThreads = 1024;   // what if numThreads>1024, numThreads<1024?
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** function ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    kernel_C <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize();

  //"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,cudaCpuDeviceId,NULL);

//error checking
    size_t err_count = 0;
    for (int i=0; i<ARRAY_SIZE; i++){
        if(idx[i] == 0) {
            if(C[i] != A[i]) {
                err_count++;
            }
        }
        else if (idx[i] == 1) {
            if(C[i] != B[i]) {
                err_count++;
            }
        }
    }
  printf("Error count(CUDA program): %zu\n", err_count);


//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;

}


Overwriting CUDA_var5.cu


In [17]:
%%bash
nvcc CUDA_var5.cu -o CUDA_var5 -Wno-deprecated-gpu-targets

In [18]:
%%bash
nvprof ./CUDA_var5

==1024382== NVPROF is profiling process 1024382, command: ./CUDA_var5


*** function ***
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Error count(CUDA program): 0


==1024382== Profiling application: ./CUDA_var5
==1024382== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.6437ms        10  364.37us  361.25us  367.97us  kernel_C(unsigned long, float*, float*, float*, int*)
      API calls:   80.48%  1.21016s         4  302.54ms  44.046us  1.20924s  cudaMallocManaged
                   17.68%  265.87ms        12  22.156ms  208.66us  75.676ms  cudaMemPrefetchAsync
                    1.38%  20.735ms         4  5.1838ms  3.7294ms  6.4861ms  cudaFree
                    0.23%  3.4470ms         1  3.4470ms  3.4470ms  3.4470ms  cudaDeviceSynchronize
                    0.15%  2.2501ms        10  225.01us  17.314us  2.0426ms  cudaLaunchKernel
                    0.05%  741.70us       114  6.5060us     180ns  343.44us  cuDeviceGetAttribute
                    0.01%  186.23us         1  186.23us  186.23us  186.23us  cuDeviceGetName
                    0.01%  108.79us         4  

# Variant 6.0 - CUDA classic MEMCPY

In [34]:
%%writefile CUDA_var6.cu
//grid stride loop + memcpy
// BFF PALANG TOH SO FAR + onting research sa websites HJSHAS
// UNFINISHED

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

//CUDA kernel
__global__
void kernel_C(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<24;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 10;

float *h_A = (float*)malloc(FLOAT_ARRAY_BYTES);
float *h_B = (float*)malloc(FLOAT_ARRAY_BYTES);
float *h_C = (float*)malloc(FLOAT_ARRAY_BYTES);
int *h_idx = (int*)malloc(INT_ARRAY_BYTES);

// ****init host array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    h_A[i] = sinf(i * 0.0005) * 100.0 + 50.0;
    h_B[i] = cosf(i * 0.0003) * 100.0 + 50.0;
  }

// device allocations
  float *d_A,*d_B,*d_C;
  int *d_idx;
  cudaMalloc(&d_A, FLOAT_ARRAY_BYTES); //we dont use cudaMallocManaged with cudaMemcpy
  cudaMalloc(&d_B, FLOAT_ARRAY_BYTES);
  cudaMalloc(&d_C, FLOAT_ARRAY_BYTES);
  cudaMalloc(&d_idx, INT_ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);

// Copy data to device
    cudaMemcpy(d_A, h_A, FLOAT_ARRAY_BYTES, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, FLOAT_ARRAY_BYTES, cudaMemcpyHostToDevice);

// Kernel launch configuration
    int threadsPerBlock = 256;
    int blocksPerGrid = (ARRAY_SIZE + threadsPerBlock - 1) / threadsPerBlock;
    
  for (size_t i=0; i<loope;i++){
    kernel_C <<<blocksPerGrid, threadsPerBlock>>> (ARRAY_SIZE, d_A, d_B, d_C, d_idx);
    cudaDeviceSynchronize();
  }

    
// Copy results back to host
    cudaMemcpy(h_C, d_C, FLOAT_ARRAY_BYTES, cudaMemcpyDeviceToHost);
    cudaMemcpy(h_idx, d_idx, INT_ARRAY_BYTES, cudaMemcpyDeviceToHost);

    // Verification loop
    int errors = 0;
    for (int i = 0; i < 100; i++) { // Check first 100 elements for brevity
        float expected_C = (h_A[i] >= h_B[i]) ? h_A[i] : h_B[i];
        int expected_idx = (h_A[i] >= h_B[i]) ? 0 : 1;
    
        if (fabs(h_C[i] - expected_C) > 1e-5 || h_idx[i] != expected_idx) {
            errors++;
        }
    }
    printf("Errors = %d", errors);

// Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    cudaFree(d_idx);

// Free host memory
    free(h_A);
    free(h_B);
    free(h_C);
    free(h_idx);

  return 0;
}

Overwriting CUDA_var6.cu


In [35]:
%%bash
nvcc CUDA_var6.cu -o CUDA_var6 -Wno-deprecated-gpu-targets

In [36]:
!./CUDA_var6

Errors = 0

In [37]:
%%bash
nvprof ./CUDA_var6

==1025053== NVPROF is profiling process 1025053, command: ./CUDA_var6


Errors = 0

==1025053== Profiling application: ./CUDA_var6
==1025053== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   82.84%  766.84ms         2  383.42ms  373.85ms  392.99ms  [CUDA memcpy DtoH]
                   16.78%  155.37ms         2  77.686ms  75.024ms  80.349ms  [CUDA memcpy HtoD]
                    0.37%  3.4546ms        10  345.46us  343.93us  349.44us  kernel_C(unsigned long, float*, float*, float*, int*)
      API calls:   60.27%  1.45820s         4  364.55ms  175.02us  1.45738s  cudaMalloc
                   39.20%  948.55ms         4  237.14ms  76.280ms  404.40ms  cudaMemcpy
                    0.18%  4.3068ms       114  37.778us     103ns  3.5793ms  cuDeviceGetAttribute
                    0.15%  3.5939ms        10  359.39us  302.02us  484.38us  cudaDeviceSynchronize
                    0.10%  2.4618ms         4  615.45us  207.75us  1.5057ms  cudaFree
                    0.08%  2.0030ms        10  200.30us  16